In [115]:
import numpy as np
from copy import copy, deepcopy

class Action:
    def __init__(self, i, j):
        self.i=i
        self.j=j
        
    def __str__(self):
        return str(self.i)+","+str(self.j)

class State:
             
  
    def __init__(self, square, n_moves=0, next_move='x'):    
        
        #state attributes
        self.square = deepcopy(square)
        self.next_move = next_move
        self.n_moves = n_moves
        self.first_visit = True
        self.value = None
        self.winner = None
        
        # tree atributes
        self.children=[]
        self.parent=None
        self.type='MAX'
    
    def get_actions(self):
        actions = []
        for i in range(3):
            for j in range(3):
                if self.square[i][j] == b'-':
                    actions.append(Action(i,j))
        return actions
    
    def transition(self, action):
        s = copy(self)
        #deepcopy(self)
        s.square[action.i][action.j] = self.next_move
        if self.next_move == 'x': s.next_move='o'
        else: s.next_move = 'x'
        s.n_moves += 1      
        if s.has_winner(action.i,action.j):
            if self.next_move=='x': s.winner='x'
            else: s.winner='o'
        
        if self.type=='MIN': s.type='MAX'
        else: s.type='MIN'
            
        return s
    
    def add_child(self, child):
        self.children.append(child)
        child.parent=self
    
    def eval(self, next_move):
        if self.winner!=None:
            if self.winner==next_move: return 1
            else: return -1
        elif self.n_moves==9: return 0
        
        elif len(self.children)>0:
            max_value=-1000
            min_value=1000
            for c in self.children:
                max_value = max(c.value,max_value)
                min_value = min(c.value,min_value)
                
            if self.type=="MAX": return max_value
            else: return min_value
                
        else: return None
            
           
        
    def has_winner(self, ii, jj):
        symbol = self.square[ii][jj]
        end = True
        for j in range(0,3):
            if symbol != self.square[ii][j]:
                end=False; break
        if end: return True
                
        end = True
        for i in range(0,3):
            if symbol != self.square[i][jj]:
                end=False; break
        if end: return True 
        
        if(ii==jj):
            end = True
            for i in range(0,3):
                j=i
                if symbol != self.square[i][j]:
                    end=False; break
            if end: return True   

        if jj==2-ii:
            end = True
            for i in range(0,3):
                j=2-i
                if symbol != self.square[i][j]:
                    end=False; break
            if end: return True   
        
        return False
    
    def __copy__(self):
        return type(self)(self.square, self.n_moves, self.next_move)


In [116]:
square = np.chararray((3, 3)); square[:] = '-'  
s0=State(square)
s1=s0.transition(Action(1,1))
s2=s1.transition(Action(0,0))
s3=s2.transition(Action(0,1))
s4=s3.transition(Action(2,1))
#s5=s4.transition(Action(2,1))

In [117]:
print(s0.square,s0.type,"\n")
print(s1.square,s1.type,"\n")
print(s2.square,s2.type,"\n")
print(s3.square,s3.type,"\n")

[[b'-' b'-' b'-']
 [b'-' b'-' b'-']
 [b'-' b'-' b'-']] MAX 

[[b'-' b'-' b'-']
 [b'-' b'x' b'-']
 [b'-' b'-' b'-']] MIN 

[[b'o' b'-' b'-']
 [b'-' b'x' b'-']
 [b'-' b'-' b'-']] MAX 

[[b'o' b'x' b'-']
 [b'-' b'x' b'-']
 [b'-' b'-' b'-']] MIN 



In [118]:
#post-order
def minimax(state): 
    S=[]
    S.append(state)
    i = 0
    while len(S)>0:
        i += 1
        s = S[-1]
        if s.first_visit:
            s.first_visit=False
            actions = s.get_actions()
            for a in actions:
                s_child=s.transition(a)
                s.add_child(s_child)
                S.append(s_child)
        else: 
            s.value = s.eval(state.next_move)
            S.pop()
    print("iterations: ",i)


In [119]:
def best_child(state):
    max_value = -10
    best_child = None
    for c in state.children:
        if c.value>max_value:
            best_child = c
            max_value=c.value
    return best_child 

In [120]:
s3.square

chararray([[b'o', b'x', b'-'],
           [b'-', b'x', b'-'],
           [b'-', b'-', b'-']], dtype='|S1')

In [122]:
s=copy(s1)
minimax(s)
s4=best_child(s)
s4.square, s4.value

iterations:  219202


(chararray([[b'o', b'-', b'-'],
            [b'-', b'x', b'-'],
            [b'-', b'-', b'-']], dtype='|S1'), 0)

In [99]:
s=s4.copy()
minimax(s)
s5=best_child(s)
s5.square

iterations:  27400


chararray([[b'o', b'x', b'-'],
           [b'-', b'x', b'-'],
           [b'-', b'-', b'-']], dtype='|S1')

In [68]:
s=s5.copy()
minimax(s)
s6=best_child(s)
s6.square

chararray([[b'o', b'x', b'-'],
           [b'x', b'x', b'o'],
           [b'-', b'o', b'-']], dtype='|S1')

In [69]:
s=s6.copy()
minimax(s)
s7=best_child(s)
s7.square

chararray([[b'o', b'x', b'x'],
           [b'x', b'x', b'o'],
           [b'-', b'o', b'-']], dtype='|S1')

In [70]:
s=s7.copy()
minimax(s)
s8=best_child(s)
s8.square

chararray([[b'o', b'x', b'x'],
           [b'x', b'x', b'o'],
           [b'o', b'o', b'-']], dtype='|S1')

In [223]:
s=s.transition(Action(2,0))
s.square

chararray([[b'o', b'x', b'x'],
           [b'x', b'x', b'o'],
           [b'o', b'o', b'-']], dtype='|S1')

In [224]:
s=minimax(s)
s.square

chararray([[b'o', b'x', b'x'],
           [b'x', b'x', b'o'],
           [b'o', b'o', b'x']], dtype='|S1')